# gerekli kütüphaneler

In [1]:
# uyarı ayarı
import warnings
warnings.filterwarnings("ignore")

# veri işleme
import pandas as pd
import numpy as np

# istatistik
import scipy as sc
import hypothetical
import pingouin
import statsmodels as sm

# veri görselleştirme
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from IPython.display import HTML, display

# kütüphane ayarları
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('mode.chained_assignment', None)

# verinin çalışma ortamına alınması

2009-2011 dönemi verileri **veri** isimli değişken olarak çalışma ortamına alınır:

In [2]:
veri = pd.read_csv("dataset/online_retail_2.csv")

In [3]:
print(veri.shape)
veri.head()

(1067371, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
veri.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1067371 non-null  object 
 1   StockCode    1067371 non-null  object 
 2   Description  1062989 non-null  object 
 3   Quantity     1067371 non-null  int64  
 4   InvoiceDate  1067371 non-null  object 
 5   Price        1067371 non-null  float64
 6   Customer ID  824364 non-null   float64
 7   Country      1067371 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 65.1+ MB


# veri hazırlığı

**gözlemler:**

veride değişkenlerin birtakım rasyonel olmayan durumları olduğunu gözlemledik:

- **değişkenler:** 
    - <font color="red"> problem </font> &rarr; <font color="green"> gereken aksiyon </font> <font color="blue">{ neden }</font>
- **StockCode:** 
    - <font color="red"> tutarsız veri tipi </font> &rarr; <font color="green"> tamsayı dönüşümü </font> <font color="blue">{ veri doğru olmayan formatla kaydedilmiş }</font>   
- **Description:** 
    - <font color="red"> kayıp bilgi </font> &rarr; <font color="green"> kayıtların atılması </font> <font color="blue">{ kayıp frekansı düşük. analizimizi etkilemez. }</font>
- **InvoiceDate:** 
    - <font color="red"> tutarsız veri tipi </font> &rarr; <font color="green"> tarih dönüşümü </font> <font color="blue">{ veri doğru olmayan formatla kaydedilmiş }</font>
- **Customer ID:** 
    - <font color="red"> kayıp bilgi </font> &rarr; <font color="green"> kayıtların atılması </font> <font color="blue">{ kayıp frekansı düşük. analizimizi etkilemez. }</font>
    - <font color="red"> tutarsız veri tipi </font> &rarr; <font color="green"> tamsayı dönüşümü </font> <font color="blue">{ veri doğru olmayan formatla kaydedilmiş }</font>  

# kayıp verilerin tanımlanması ve çözümlenmesi

kayıp veri sorununu çözmeden önce:

In [5]:
kayıp_veri = pd.DataFrame(index = veri.columns.values)
kayıp_veri['NullFrequency'] = veri.isnull().sum().values
yüzde = veri.isnull().sum().values/veri.shape[0]
kayıp_veri['MissingPercent'] = np.round(yüzde, decimals = 4) * 100
kayıp_veri.transpose()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
NullFrequency,0.0,0.0,4382.00,0.0,0.0,0.0,243007.00,0.0
MissingPercent,0.0,0.0,0.41,0.0,0.0,0.0,22.77,0.0


**gözlemler:**

- **değişkenler:** 
    - <font color="red"> problem </font> &rarr; <font color="green"> gereken aksiyon </font> <font color="blue">{ neden }</font>
- **Description:**
  - <font color="red"> kayıp veri (4382)</font> &rarr; <font color="green">kayıtları at</font> <font color="blue">{oran çok düşük ve analizimizi etkilemez.}</font>
- **Customer ID:**
  - <font color="red">kayıp veri (243007)</font> &rarr; <font color="green">kayıtları at</font> <font color="blue">{segmentasyon modelinde potansiyel bir değişken, ama modelde kayıp değer kullanamayız.}</font>

aksiyonlar:

In [6]:
önceki_veri_boyutu = veri.shape
print('veri boyutu [önce]:', önceki_veri_boyutu)

veri.dropna(axis = 0, subset = ['Description', 'Customer ID'], inplace = True)

sonraki_veri_boyutu = veri.shape
print('veri boyutu [sonra]:', sonraki_veri_boyutu)

atılan_sayı  = önceki_veri_boyutu[0] - sonraki_veri_boyutu[0]
atılan_oran  = np.round(atılan_sayı / önceki_veri_boyutu[0], decimals = 2) * 100
print('atılan oran:', atılan_oran, '%')

veri boyutu [önce]: (1067371, 8)
veri boyutu [sonra]: (824364, 8)
atılan oran: 23.0 %


yaptıklarımızı doğrulayalım:

In [7]:
kayıp_veri = pd.DataFrame(index = veri.columns.values)
kayıp_veri['NullFrequency'] = veri.isnull().sum().values
yüzde = veri.isnull().sum().values/veri.shape[0]
kayıp_veri['MissingPercent'] = np.round(yüzde, decimals = 4) * 100
kayıp_veri.transpose()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
NullFrequency,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MissingPercent,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


böylece kayıp değerleri **başarılı bir şekilde eledik.**

# gereksiz verilerin tanımlanması ve çözümlenmesi

çoklayan kayıtların kontrolü:

In [8]:
print('veride çoklayan kayıt var mı?:', veri.duplicated().any())
print('çoklayan kayıt sayısı:', veri.duplicated().sum())

veride çoklayan kayıt var mı?: True
çoklayan kayıt sayısı: 26479


**gözlemler:**

- buna göre veride **26.479 çoklayan** satır **bulunuyor**.
- bu kayıtları da analizlerimizde işe yaramayacağından **atıyoruz.** 

aksiyonlar:

In [9]:
önceki_veri_boyutu = veri.shape
print('veri boyutu [önce]:', önceki_veri_boyutu)

veri.drop_duplicates(inplace = True)

sonraki_veri_boyutu = veri.shape
print('veri boyutu [sonra]:', sonraki_veri_boyutu)

atılan_sayı  = önceki_veri_boyutu[0] - sonraki_veri_boyutu[0]
atılan_oran  = np.round(atılan_sayı / önceki_veri_boyutu[0], decimals = 2) * 100
print('atılan oran:', atılan_oran, '%')

veri boyutu [önce]: (824364, 8)
veri boyutu [sonra]: (797885, 8)
atılan oran: 3.0 %


yaptıklarımızı doğrulayalım:

In [10]:
print('veride çoklayan kayıt var mı?:', veri.duplicated().any())
print('çoklayan kayıt sayısı:', veri.duplicated().sum())

veride çoklayan kayıt var mı?: False
çoklayan kayıt sayısı: 0


böylece çoklayan kayıtları da **başarılı bir şekilde eledik.**

# tutarsız veri tiplerinin tanımlanması ve çözümlenmesi

In [11]:
veri_tipleri = pd.DataFrame(data = veri.dtypes, columns = ['Type'])
veri_tipleri.transpose()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
Type,object,object,object,int64,object,float64,float64,object


**gözlemler:**

- **tutarsız veri:** 
    - <font color="red">mevcut değişken tipi</font> &rarr; <font color = "green">beklenen değişken tipi</font>
- **Invoice:**
    - <font color="red">object</font> &rarr; <font color = "green">integer</font>
- **InvoiceDate:**
    - <font color="red">object</font> &rarr; <font color = "green">DateTime</font>
- **Customer ID:**
    - <font color="red">float</font> &rarr; <font color = "green">integer</font>

aksiyonlar:

In [12]:
#veri['Invoice']       = veri['Invoice'].astype(np.int64)
veri['InvoiceDate']   = pd.to_datetime(veri['InvoiceDate'])
veri['Customer ID']   = veri['Customer ID'].astype(np.int64)

* InvoiceNo: Fatura numarası. Nominal. Her işleme benzersiz şekilde atanan 6 basamaklı bir sayı. Bu kod 'c' harfiyle başlıyorsa, bir iptal olduğunu gösterir.

In [13]:
veri.sort_values(by='Invoice', ascending=False).head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1067178,C581569,20979,36 PENCILS TUBE RED RETROSPOT,-5,2011-12-09 11:58:00,1.25,17315,United Kingdom
1067177,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,2011-12-09 11:58:00,1.25,17315,United Kingdom
1067176,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,2011-12-09 11:57:00,10.95,15311,United Kingdom
1067002,C581499,M,Manual,-1,2011-12-09 10:28:00,224.69,15498,United Kingdom
1065909,C581490,22178,VICTORIAN GLASS HANGING T-LIGHT,-12,2011-12-09 09:57:00,1.95,14397,United Kingdom


In [14]:
print("'C' ile başlayıp iptal edilmemiş işlem var mı?",((veri[veri['Invoice'].str.startswith("C",na=False)]['Quantity']) > 0).any())
print("'C' ile başlamayıp iptal edilmiş işlem var mı?", ((veri[veri['Invoice'].str.startswith("C",na=False) == False]['Quantity']) < 0).any())

'C' ile başlayıp iptal edilmemiş işlem var mı? False
'C' ile başlamayıp iptal edilmiş işlem var mı? False


In [15]:
veri = veri[veri['Invoice'].str.startswith("C",na=False) == False]
veri['Invoice'] = veri['Invoice'].astype(np.int64)

yaptıklarımızı kontrol edelim:

In [16]:
veri_tipleri = pd.DataFrame(data = veri.dtypes, columns = ['Type'])
veri_tipleri.transpose()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
Type,int64,object,object,int64,datetime64[ns],float64,int64,object


# yeni değişkenlerin oluşturulması

toplam fiyat değişkeni:

In [17]:
veri['TotalPrice'] = veri['Price']*veri['Quantity']

bölge değişkeni:

In [18]:
# ülke grupları
avrupa_ülkeleri = ['Austria', 'Belgium', 'Cyprus', 'Czech Republic', 'Denmark',
                   'EIRE', 'European Community', 'Finland', 'France', 'Germany',
                   'Greece', 'Iceland','Italy', 'Lithuania', 'Malta', 'Netherlands',
                   'Norway', 'Poland', 'Portugal', 'Spain', 'Sweden', 'Switzerland',
                   'United Kingdom', 'Channel Islands']

amerika_ülkeleri = ['Canada', 'USA', 'Brazil', 'Bermuda']

asya_ülkeleri = ['Bahrain','Hong Kong', 'Japan', 'Saudi Arabia', 'Singapore', 'Thailand', 'United Arab Emirates']

# ülke grupları fonksiyon
def ülke_grubu(row):
    global avrupa_ülkeleri
    global amerika_ülkeleri
    global asya_ülkeleri
    
    if row['Country'] in avrupa_ülkeleri:
        return "Europe"
    elif row['Country'] in amerika_ülkeleri:
        return "America"
    elif row['Country'] in asya_ülkeleri:
        return "Asia"
    else:
        return "Other"

In [19]:
veri = veri.assign(CountryGroup=veri.apply(ülke_grubu, axis=1))

son olarak veriye bakalım:

In [20]:
print(veri.shape)
veri.head()

(779495, 10)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice,CountryGroup
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,83.4,Europe
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,Europe
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,81.0,Europe
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,100.8,Europe
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,30.0,Europe


In [21]:
veri.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 779495 entries, 0 to 1067370
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Invoice       779495 non-null  int64         
 1   StockCode     779495 non-null  object        
 2   Description   779495 non-null  object        
 3   Quantity      779495 non-null  int64         
 4   InvoiceDate   779495 non-null  datetime64[ns]
 5   Price         779495 non-null  float64       
 6   Customer ID   779495 non-null  int64         
 7   Country       779495 non-null  object        
 8   TotalPrice    779495 non-null  float64       
 9   CountryGroup  779495 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(3), object(4)
memory usage: 65.4+ MB


In [22]:
print(f"minimum alışveriş tarihi: {veri['InvoiceDate'].min()}")
print(f"minimum alışveriş tarihi: {veri['InvoiceDate'].max()}")

minimum alışveriş tarihi: 2009-12-01 07:45:00
minimum alışveriş tarihi: 2011-12-09 12:50:00
